In [1]:
# ==========================================
# CELL 1: CÀI ĐẶT VÀ IMPORT
# ==========================================

# 1. Cài đặt thư viện (Chỉ cần chạy 1 lần)
!pip install chefboost

# 2. Import các thư viện cần thiết
import pandas as pd
import numpy as np
from chefboost import Chefboost as cb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Tắt cảnh báo để output gọn gàng hơn
warnings.filterwarnings('ignore')

In [2]:
# ==========================================
# CELL 2: TIỀN XỬ LÝ DỮ LIỆU (ĐÃ SỬA)
# ==========================================

# 1. Load dữ liệu
try:
    df = pd.read_csv('/kaggle/input/airline-passenger-satisfaction/train.csv')
except FileNotFoundError:
    df = pd.read_csv('train.csv')

# 2. Làm sạch (Cleaning)
drop_cols = ['id', 'Unnamed: 0']
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
df = df.dropna()

# --- SỬA ĐỔI QUAN TRỌNG: TẠM BỎ BƯỚC LỌC OUTLIER ---
# Lý do: Việc gom nhóm (Binning) ở dưới đã giúp xử lý các giá trị lớn rồi.
# Nếu lọc outlier bằng IQR ở đây sẽ bị mất hết các trường hợp 'Delayed' và 'Very delayed'.
# (Đã comment lại đoạn code lọc nhiễu)
# for col in ["Arrival Delay in Minutes", "Departure Delay in Minutes"]:
#     Q1 = df[col].quantile(0.25)
#     Q3 = df[col].quantile(0.75)
#     IQR = Q3 - Q1
#     lower = Q1 - 1.5 * IQR
#     upper = Q3 + 1.5 * IQR
#     df = df[(df[col] >= lower) & (df[col] <= upper)]

# 3. Gom nhóm dữ liệu (Binning)

# a. Tuổi
bins_age = [0, 19, 29, 39, 49, 59, 120]
labels_age = ['<20', '20-29', '30-39', '40-49', '50-59', '60+']
df['Age'] = pd.cut(df['Age'], bins=bins_age, labels=labels_age)

# b. Thời gian trễ
# (-1 để bao gồm cả 0)
bins_delay = [-1, 0, 5, 15, 30, 100000]
labels_delay = ['On time', 'Slightly delayed', 'Moderately delayed', 'Delayed', 'Very delayed']

df['Arrival Delay in Minutes'] = pd.cut(df['Arrival Delay in Minutes'], bins=bins_delay, labels=labels_delay)
df['Departure Delay in Minutes'] = pd.cut(df['Departure Delay in Minutes'], bins=bins_delay, labels=labels_delay)

# c. Khoảng cách bay
bins_dist = [0, 500, 1000, 1500, 2000, 2500, df['Flight Distance'].max() + 1]
labels_dist = ['0-500', '501-1000', '1001-1500', '1501-2000', '2001-2500', '2500+']
df['Flight Distance'] = pd.cut(df['Flight Distance'], bins=bins_dist, labels=labels_dist)

# 4. Mã hóa (Encoding)
cat_cols = ['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
            'Departure Delay in Minutes', 'Arrival Delay in Minutes',
            'satisfaction', 'Flight Distance']

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = df[col].astype(str)
    # Fit encoder trên toàn bộ dữ liệu hiện có
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

print("Dữ liệu sau khi tiền xử lý (Đã giữ lại các nhóm trễ):")
print(df.head())

Dữ liệu sau khi tiền xử lý (Đã giữ lại các nhóm trễ):
   Gender  Customer Type  Age  Type of Travel  Class  Flight Distance  \
0       1              0    5               1      2                0   
1       1              1    0               0      0                0   
2       0              0    0               0      0                1   
3       0              0    0               0      0                5   
4       1              0    4               0      0                0   

   Inflight wifi service  Departure/Arrival time convenient  \
0                      3                                  4   
1                      3                                  2   
2                      2                                  2   
3                      2                                  5   
4                      3                                  3   

   Ease of Online booking  Gate location  ...  Inflight entertainment  \
0                       3              1  ...          

In [3]:
# ==========================================
# CELL 3: CHIA TẬP TRAIN / TEST
# ==========================================

# 1. Tách biến đầu vào (X) và nhãn mục tiêu (y)
X = df.drop(columns=['satisfaction'])
y = df['satisfaction']

# 2. Chia tập dữ liệu: 80% Train, 20% Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Chuẩn bị DataFrame cho Chefboost
# Chefboost yêu cầu nhãn mục tiêu phải nằm trong cột cuối cùng của DataFrame huấn luyện
df_train = X_train.copy()
df_train['satisfaction'] = y_train

# Chefboost yêu cầu target phải ở dạng string (object) để hiểu là bài toán phân loại
df_train['satisfaction'] = df_train['satisfaction'].astype(str)

print(f"Kích thước tập Train: {df_train.shape}")
print(f"Kích thước tập Test: {X_test.shape}")

Kích thước tập Train: (82875, 23)
Kích thước tập Test: (20719, 22)


In [4]:
# ==========================================
# CELL 4: HUẤN LUYỆN MÔ HÌNH ID3
# ==========================================

# Cấu hình thuật toán
config = {
    'algorithm': 'ID3',
    'enableParallelism': True # Bật xử lý song song để nhanh hơn
}

print("Đang huấn luyện mô hình ID3... (Vui lòng đợi)")
# Bắt đầu train
model = cb.fit(df_train, config=config, target_label='satisfaction')

Đang huấn luyện mô hình ID3... (Vui lòng đợi)
25-11-27 16:28:07 - [INFO]: 2 CPU cores will be allocated in parallel running
25-11-27 16:28:07 - ID3 tree is going to be built...
25-11-27 16:28:56 - -------------------------
25-11-27 16:28:56 - finished in 48.35418701171875 seconds
25-11-27 16:29:05 - -------------------------
25-11-27 16:29:05 - Evaluate train set
25-11-27 16:29:05 - -------------------------
25-11-27 16:29:05 - Accuracy: 89.55052790346907% on 82875 instances
25-11-27 16:29:05 - Labels: ['1' '0']
25-11-27 16:29:05 - Confusion matrix: [[30781, 3608], [5052, 43434]]
25-11-27 16:29:05 - Precision: 89.5083%, Recall: 85.9013%, F1: 87.6677%


In [5]:
# ==========================================
# CELL 5: ĐÁNH GIÁ MÔ HÌNH
# ==========================================

y_pred_list = []
# Lấy danh sách tên cột theo đúng thứ tự lúc train
feature_columns = X_train.columns.tolist()

print("Đang dự đoán trên tập Test...")

# Chefboost dự đoán từng dòng (row-by-row prediction)
for index, row in X_test.iterrows():
    instance = row[feature_columns].tolist()
    prediction = cb.predict(model, instance)
    y_pred_list.append(prediction)

# Chuyển kết quả dự đoán về dạng số nguyên để so sánh với y_test
y_pred_numeric = [int(p) for p in y_pred_list]

# In báo cáo kết quả
print("\n" + "="*30)
print("KẾT QUẢ ĐÁNH GIÁ (ID3)")
print("="*30)
print("Accuracy (Độ chính xác):", accuracy_score(y_test, y_pred_numeric))
print("\nClassification Report:\n", classification_report(y_test, y_pred_numeric))

Đang dự đoán trên tập Test...

KẾT QUẢ ĐÁNH GIÁ (ID3)
Accuracy (Độ chính xác): 0.8972440754862686

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.92      0.91     11655
           1       0.90      0.86      0.88      9064

    accuracy                           0.90     20719
   macro avg       0.90      0.89      0.90     20719
weighted avg       0.90      0.90      0.90     20719



In [6]:
# ==========================================
# CELL 6: DỰ ĐOÁN MẪU MỚI (DEMO)
# ==========================================

# Dữ liệu mẫu (Lưu ý: Phải nhập đúng các Nhóm/Labels đã chia ở Cell 2)
sample_input = {
    'Gender': 'Male',
    'Customer Type': 'Loyal Customer',
    'Age': '20-29',                # Đã qua binning
    'Type of Travel': 'Business travel',
    'Class': 'Business',
    'Flight Distance': '501-1000', # Đã qua binning
    'Inflight wifi service': 3,
    'Departure/Arrival time convenient': 4,
    'Ease of Online booking': 3,
    'Gate location': 1,
    'Food and drink': 5,
    'Online boarding': 3,
    'Seat comfort': 5,
    'Inflight entertainment': 5,
    'On-board service': 4,
    'Leg room service': 3,
    'Baggage handling': 4,
    'Checkin service': 4,
    'Inflight service': 5,
    'Cleanliness': 5,
    'Departure Delay in Minutes': 'Delayed', # Đã qua binning
    'Arrival Delay in Minutes': 'Delayed'    # Đã qua binning
}

# Mã hóa dữ liệu mẫu (Sử dụng lại label_encoders đã học ở Cell 2)
encoded_sample = []
for col in X_train.columns:
    val = sample_input.get(col)
    # Nếu cột này nằm trong danh sách đã mã hóa thì transform
    if col in label_encoders:
        # Chuyển giá trị chữ sang số tương ứng
        val_encoded = label_encoders[col].transform([str(val)])[0]
        encoded_sample.append(val_encoded)
    else:
        # Các cột điểm số (1-5) giữ nguyên
        encoded_sample.append(val)

# Dự đoán
pred_val = cb.predict(model, encoded_sample)

# Giải mã kết quả (Inverse Transform để biết 0/1 là Neutral hay Satisfied)
final_result = label_encoders['satisfaction'].inverse_transform([int(pred_val)])[0]

print("\n" + "="*30)
print("DỰ ĐOÁN MẪU ĐƠN LẺ")
print("="*30)
print(f"Input: {sample_input}")
print(f"Kết quả dự đoán (Encoded): {pred_val}")
print(f"Kết quả cuối cùng: {final_result}")


DỰ ĐOÁN MẪU ĐƠN LẺ
Input: {'Gender': 'Male', 'Customer Type': 'Loyal Customer', 'Age': '20-29', 'Type of Travel': 'Business travel', 'Class': 'Business', 'Flight Distance': '501-1000', 'Inflight wifi service': 3, 'Departure/Arrival time convenient': 4, 'Ease of Online booking': 3, 'Gate location': 1, 'Food and drink': 5, 'Online boarding': 3, 'Seat comfort': 5, 'Inflight entertainment': 5, 'On-board service': 4, 'Leg room service': 3, 'Baggage handling': 4, 'Checkin service': 4, 'Inflight service': 5, 'Cleanliness': 5, 'Departure Delay in Minutes': 'Delayed', 'Arrival Delay in Minutes': 'Delayed'}
Kết quả dự đoán (Encoded): 0
Kết quả cuối cùng: neutral or dissatisfied
